In [ ]:
!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import sys
sys.path.insert(0, "../input/weightedboxesfusion")
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torchvision import transforms
from matplotlib import pyplot as plt
from torchvision import transforms
import torchvision 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torch.utils.data import Dataset,DataLoader
import glob
import albumentations as A
import cv2
from albumentations.pytorch.transforms import ToTensorV2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import ensemble_boxes
from ensemble_boxes import *
import effdet
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchEval
from effdet.efficientdet import HeadNet

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def get_model_d7(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d7')
    model = EfficientDet(config, pretrained_backbone=False)
    config.num_classes = 1
    config.image_size=512
    model.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = DetBenchEval(model, config)
    model.to(device)
    model.eval();
    return model

def get_model_d5(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    model = EfficientDet(config, pretrained_backbone=False)
    config.num_classes = 1
    config.image_size=512
    model.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = DetBenchEval(model, config)
    model.to(device)
    model.eval();
    return model

In [ ]:
eff_d7_33 = "../input/d7effiecient-33e/cp33 (1).pt"
eff_d7_26e= "../input/d726eefficient/cp26.pt"
# resnet101_50e = "/content/gdrive/My Drive/CV/פרויקטון/anna_saved_models/saved- RCNN-1024/אימון2 - לוס 0.68:(/cp49.pt"
eff_d5_21= "../input/d520eefficient/cp21.pt"
eff_D7_39e= "../input/d7efficient-39e/efficient-D7-39e.pt.pth"

In [ ]:
models = [
    get_model_d7(eff_d7_33),
    get_model_d7(eff_d7_26e),
    get_model_d7(eff_D7_39e),
#     get_model_d5(eff_d5_21)

    # load_resnet101_model(resnet101_33e),
    # load_resnet101_model(resnet101_50e)
]

In [ ]:
def get_valid_transforms():
    return A.Compose([
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [ ]:
DATA_ROOT_PATH = "../input/global-wheat-detection/test"

class DatasetRetriever(Dataset):

    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
dataset = DatasetRetriever(np.array([path.split('/')[-1][:-4] for path in glob.glob(f'{DATA_ROOT_PATH}/*.jpg')]),get_valid_transforms())

def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=1,
    drop_last=False,
    collate_fn=collate_fn
)

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

In [ ]:
class BaseWheatTTA:
    """ author: @shonenkov """
    image_size = 512

    def augment(self, image):
        raise NotImplementedError
    
    def batch_augment(self, images):
        raise NotImplementedError
    
    def deaugment_boxes(self, boxes):
        raise NotImplementedError

class TTAHorizontalFlip(BaseWheatTTA):
    """ author: @shonenkov """

    def augment(self, image):
        return image.flip(1)
    
    def batch_augment(self, images):
        return images.flip(2)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [1,3]] = self.image_size - boxes[:, [3,1]]
        return boxes

class TTAVerticalFlip(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return image.flip(2)
    
    def batch_augment(self, images):
        return images.flip(3)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [0,2]] = self.image_size - boxes[:, [2,0]]
        return boxes
    
class TTARotate90(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return torch.rot90(image, 1, (1, 2))

    def batch_augment(self, images):
        return torch.rot90(images, 1, (2, 3))
    
    def deaugment_boxes(self, boxes):
        res_boxes = boxes.copy()
        res_boxes[:, [0,2]] = self.image_size - boxes[:, [1,3]]
        res_boxes[:, [1,3]] = boxes[:, [2,0]]
        return res_boxes
    
class TTARotate180(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return torch.rot90(image, 2, (1, 2))

    def batch_augment(self, images):
        return torch.rot90(images, 2, (2, 3))
    
    def deaugment_boxes(self, boxes):
        boxes[:, [0,1,2,3]] = self.image_size - boxes[:, [2,3,0,1]]
        return boxes
    
class TTARotate270(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return torch.rot90(image, 3, (1, 2))

    def batch_augment(self, images):
        return torch.rot90(images, 3, (2, 3))
    
    def deaugment_boxes(self, boxes):
        res_boxes = boxes.copy()
        res_boxes[:, [0,2]] = boxes[:, [1,3]]
        res_boxes[:, [1,3]] = self.image_size - boxes[:, [2,0]]
        return res_boxes
    
class TTACompose(BaseWheatTTA):
    """ author: @shonenkov """
    def __init__(self, transforms):
        self.transforms = transforms
        
    def augment(self, image):
        for transform in self.transforms:
            image = transform.augment(image)
        return image
    
    def batch_augment(self, images):
        for transform in self.transforms:
            images = transform.batch_augment(images)
        return images
    
    def prepare_boxes(self, boxes):
        result_boxes = boxes.copy()
        result_boxes[:,0] = np.min(boxes[:, [0,2]], axis=1)
        result_boxes[:,2] = np.max(boxes[:, [0,2]], axis=1)
        result_boxes[:,1] = np.min(boxes[:, [1,3]], axis=1)
        result_boxes[:,3] = np.max(boxes[:, [1,3]], axis=1)
        return result_boxes
    
    def deaugment_boxes(self, boxes):
        for transform in self.transforms[::-1]:
            boxes = transform.deaugment_boxes(boxes)
        return self.prepare_boxes(boxes)

In [ ]:
from itertools import product

tta_transforms = []
for tta_combination in product([TTAHorizontalFlip(), None], 
                               [TTAVerticalFlip(), None],
                               [TTARotate90(), None]):
    tta_transforms.append(TTACompose([tta_transform for tta_transform in tta_combination if tta_transform]))

In [ ]:
# def make_tta_predictions(model, images, score_threshold=0.25):
#     model.eval()
#     model.to(device)
#     with torch.no_grad():
#         images = torch.stack(images).float().cuda()
#         predictions = []
#         for tta_transform in tta_transforms:
#             result = []
#             det = model(tta_transform.batch_augment(images.clone()), img_size =  torch.tensor([images[0].shape[-2:]] * 1, dtype=torch.float).to(device), img_scales= torch.tensor([1]*images.shape[0]).float().cuda())

#             for i in range(images.shape[0]):
#                 boxes = det[i].detach().cpu().numpy()[:,:4]    
#                 scores = det[i].detach().cpu().numpy()[:,4]
#                 indexes = np.where(scores > score_threshold)[0]
#                 boxes = boxes[indexes]
#                 boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
#                 boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
#                 boxes = tta_transform.deaugment_boxes(boxes.copy())
#                 result.append({
#                     'boxes': boxes,
#                     'scores': scores[indexes],
#                 })
#             predictions.append(result)
#     return predictions

In [ ]:
import ensemble_boxes
from ensemble_boxes import *

# def make_ensemble_predictions(images):
#     images = list(image.to(device) for image in images)    
#     result = []
#     for model in models:
#         predictions = make_tta_predictions(model, images)
#         with torch.no_grad():
#             for i in range(len(images)):
#                 boxes, scores, labels = run_wbf(predictions, image_index=i)
#                 outputs = {'boxes': boxes,'labels': labels, 'scores':scores }
#                 result.append([outputs])
#                 torch.cuda.empty_cache()

#     return result

def make_ensemble_predictions(images):
    # images = list(image.to(device) for image in images)
    images = torch.stack(images).cuda().float()    
    result = []
    for model in models:
        with torch.no_grad():
            model.eval()
#             x= torch.tensor([1]*images.shape[0]).float().cuda()
#             y = torch.tensor([images[0].shape[-2:]] * 1, dtype=torch.float).to(device)
            outputs = model(images, torch.tensor([1.0] * 1, dtype=torch.float).to(device))
            results = []
            for i in range(images.shape[0]):
                boxes = outputs[i].detach().cpu().numpy()[:,:4]    
                scores = outputs[i].detach().cpu().numpy()[:,4]
                indexes = np.where(scores > 0.23)[0]
                boxes = boxes[indexes]
                boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                results.append({
                    'boxes': boxes,
                    'scores': scores[indexes],
                })
            result.append(results)
            torch.cuda.empty_cache()
    return result

def run_wbf(predictions, image_index, image_size=512, iou_thr=0.3, skip_box_thr=0.25,score_threshold= 0.28, weights=None ):
    boxes = [(prediction[image_index]['boxes']/(image_size)).tolist() for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist() for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).astype(int).tolist() for prediction in predictions]
    boxes, scores, labels = ensemble_boxes.ensemble_boxes_wbf.weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    indexes = np.where(scores > score_threshold)[0]
    boxes = boxes[indexes]    
    scores = scores[indexes]
    boxes = boxes*(image_size)
    return boxes, scores, labels

In [ ]:
for j, (images, image_ids) in enumerate(data_loader):
    break
predictions = make_ensemble_predictions(images)

i = 0
sample = images[i].permute(1,2,0).cpu().numpy()
boxes, scores, labels = run_wbf(predictions, image_index=i, weights = [2,2,4])
boxes = boxes.round().astype(np.int32).clip(min=0, max=1023)

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for j,box in enumerate(boxes):
    cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 2)
    score=str(float("{:.3f}".format(scores[j])))
    
#     cv2.putText(
#                 sample,
#                 score,
#                 org=(int(box[0]), int(box[1] )), # bottom left
#                 fontFace=cv2.FONT_HERSHEY_PLAIN,
#                 fontScale=1.5,
#                 color=(1,0, 0),
#                 thickness=2
#             )
ax.set_axis_off()
ax.imshow(sample);

In [ ]:
validation_image_precisions = []
iou_thresholds = [x for x in np.arange(0.5, 0.76, 0.05)]
# model.eval()
# model.to(device)

results = []

for images, image_ids in data_loader:    
    predictions = make_ensemble_predictions(images)
    for i, image in enumerate(images):
        boxes, scores, labels = run_wbf(predictions, image_index=i, weights = [2,2,4])
        boxes = (boxes*2).astype(np.int32).clip(min=0, max=1023)
        image_id = image_ids[i]

        #plot the images
#         sample = images[i].permute(1,2,0).cpu().numpy()
#         sample = cv2.resize(sample,(1024,1024))
#         fig, ax = plt.subplots(1, 1, figsize=(6, 6))

#         for j,box in enumerate(boxes):
#             cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 3)
#             score=str(float("{:.3f}".format(scores[j])))

#             cv2.putText(
#               sample,
#               score,
#               org=(int(box[0]), int(box[1] )), # bottom left
#               fontFace=cv2.FONT_HERSHEY_PLAIN,
#               fontScale=3,
#               color=(1,0, 0),
#               thickness=3
#             )
#         ax.set_axis_off()
#         ax.imshow(sample)
        


        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }
        results.append(result)


In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)
test_df.head(10)